In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from lightgbm import LGBMClassifier
import optuna
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder

tqdm.pandas()

In [2]:
train = pd.read_csv('./data/train_ohe_te_fillna.csv')
test = pd.read_csv('./data/Test.zip')

useful_features = [
'MONTANT',
'FREQUENCE_RECH',
'REVENUE',
'ARPU_SEGMENT',
'FREQUENCE',
'DATA_VOLUME',
'ON_NET',
'ORANGE',
'TIGO',
'ZONE1',
'ZONE2',
'REGULARITY',
'FREQ_TOP_PACK',
'TENURE_CHURN_mean',
'REGION_CHURN_mean',
'REGION_0',
'REGION_1',
'REGION_2',
'REGION_3',
'REGION_4',
'REGION_5',
'REGION_6',
'REGION_7',
'REGION_8',
'REGION_9',
'REGION_10',
'REGION_11',
'REGION_12',
'REGION_13',
'TENURE_0',
'TENURE_1',
'TENURE_2',
'TENURE_3',
'TENURE_4',
'TENURE_5',
'TENURE_6',
'TENURE_7',
]

useful_cols = [
    'REGION',
    'TENURE',
    'MONTANT',
    'FREQUENCE_RECH',
    'REVENUE',
    'ARPU_SEGMENT',
    'FREQUENCE',
    'DATA_VOLUME',
    'ON_NET', 
    'ORANGE',
    'TIGO',
    'ZONE1',
    'ZONE2',
    'REGULARITY',
    'FREQ_TOP_PACK',
    'TENURE_CHURN_mean',
    'REGION_CHURN_mean'
]

cat_cols = [
    'REGION',
    'TENURE',
]

final_predictions = []
scores = []

def run(trial):
    fold = 0
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    xtrain = train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)

    ytrain = xtrain['CHURN']
    yvalid = xvalid['CHURN']

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

    model = LGBMClassifier(
        random_state=42,
        n_estimators=7000,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    score = roc_auc_score(yvalid, preds_valid)
    return score

In [3]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=10)

[I 2021-09-08 18:40:38,422] A new study created in memory with name: no-name-72ac63d9-5d4f-4419-b8e7-c183272c1362


Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[72]	valid_0's binary_logloss: 0.27382


[I 2021-09-08 18:41:03,236] Trial 0 finished with value: 0.7709722003791525 and parameters: {'learning_rate': 0.2190781226698892, 'reg_lambda': 0.0003284071493458288, 'reg_alpha': 0.004901298783146639, 'subsample': 0.4754336210716843, 'colsample_bytree': 0.34126993422447205, 'max_depth': 6}. Best is trial 0 with value: 0.7709722003791525.


Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[588]	valid_0's binary_logloss: 0.273255


[I 2021-09-08 18:41:59,445] Trial 1 finished with value: 0.7698041272707171 and parameters: {'learning_rate': 0.047477032585942915, 'reg_lambda': 3.0075761133630468, 'reg_alpha': 70.10750174715302, 'subsample': 0.13429150325394096, 'colsample_bytree': 0.5454341528843681, 'max_depth': 7}. Best is trial 1 with value: 0.7698041272707171.


...

[I 2021-09-08 19:02:48,249] Trial 7 finished with value: 0.7721984812817556 and parameters: {'learning_rate': 0.014495578147654307, 'reg_lambda': 0.09349013879849498, 'reg_alpha': 0.05678457818591098, 'subsample': 0.3647739815175761, 'colsample_bytree': 0.20000814011947332, 'max_depth': 4}. Best is trial 1 with value: 0.7698041272707171.


Training until validation scores don't improve for 300 rounds
[1000]	valid_0's binary_logloss: 0.273151
Early stopping, best iteration is:
[828]	valid_0's binary_logloss: 0.273137


[I 2021-09-08 19:03:55,489] Trial 8 finished with value: 0.7697970531958969 and parameters: {'learning_rate': 0.057192504267984716, 'reg_lambda': 0.2020682138073545, 'reg_alpha': 9.291468887449219e-07, 'subsample': 0.8824763845186918, 'colsample_bytree': 0.7382762785360262, 'max_depth': 6}. Best is trial 8 with value: 0.7697970531958969.


Training until validation scores don't improve for 300 rounds
[1000]	valid_0's binary_logloss: 0.273558
[2000]	valid_0's binary_logloss: 0.273397
Early stopping, best iteration is:
[2348]	valid_0's binary_logloss: 0.273373


[I 2021-09-08 19:06:09,894] Trial 9 finished with value: 0.7703323651531172 and parameters: {'learning_rate': 0.06372188126520278, 'reg_lambda': 1.698531773498989e-06, 'reg_alpha': 0.0057516873596360025, 'subsample': 0.4940807978973274, 'colsample_bytree': 0.4269292377321222, 'max_depth': 3}. Best is trial 8 with value: 0.7697970531958969.


In [4]:
study.best_params

{'learning_rate': 0.057192504267984716,
 'reg_lambda': 0.2020682138073545,
 'reg_alpha': 9.291468887449219e-07,
 'subsample': 0.8824763845186918,
 'colsample_bytree': 0.7382762785360262,
 'max_depth': 6}